This notebook is used to read in the text files from the data folder, and write to files that contain formatted and merged data in the stats folder.

### Import Statements

In [3]:
import json
import math
import matplotlib.pyplot as plt

# for reading shapefiles
import shapefile

# for converting longitude and latitude to UTM coordinates
import utm

# for determining what zip code a road lies in
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

### Helper Functions

In [4]:
def get_zone_conversion(latitude):
    ''' helper function for convert_to_utm. Given a latitude value, it 
        returns an approximation for how the utm coordinates change as
        the longitude crosses from UTM 17 to UTM 18.
        Adjust londelta to get a more precise answer'''
    londelta = .0000000001
    lat = latitude
    return utm.from_latlon(lat,-78 - londelta)[0] - utm.from_latlon(lat,-78 + londelta)[0]

def convert_to_utm(lonlat_coords):
    ''' Convert a list of (lon, lat) coordinates to UTM. Assumes the order is 
        longitude, then latitude. This has been customized to work for New 
        York, so the input has to be in UTM zone 17 or 18. If it is
        in 17, the coordinates are converted to be in terms of zone 18'''
    utm_coords = []
    for (lon,lat) in lonlat_coords:
        new_coords = utm.from_latlon(lat,lon)
        
        # change the x coordinates if we are in zone 17
        if new_coords[2] == 17:
            utm_coords.append([new_coords[0] - get_zone_conversion(lat), new_coords[1]])
        # otherwise leave it alone
        else:
            utm_coords.append(new_coords[:2])
    return utm_coords

def split_to_dict(s):
    ''' take a string s of the form '"a" => "b","c" => "d"',
        and return a dict {"a":"b", "c":"d"}'''
    D = {}
    split_on_comma = s.split('","')
    for pair in split_on_comma:
        key_value = pair.split('"=>"')
        if len(key_value) == 2:
            D[key_value[0]] = key_value[1]
    return D

def get_zip_code(coordinates, file_name="stats/western_ny_zip_codes.txt"):
    ''' return the zip code of the center of the given coordinates, None if
        it is not in any of them.  Assuming WNY by default.'''
    if len(coordinates) == 0:
        return None
    zip_code = None
    
    # get the average of the coordinates
    avg_x = sum([x for (x,y) in coordinates])/len(coordinates)
    avg_y = sum([y for (x,y) in coordinates])/len(coordinates)
    zc_file = open(file_name, 'r')
    for line in zc_file:
        zc = json.loads(line)
        
        # when we find the right zip code
        if Polygon(zc[1]).contains(Point([avg_x,avg_y])):
            zip_code = zc[0]
            break
    zc_file.close()
    return zip_code


def same_name(osm_name, nys_name):
    ''' return true if osm_name and nys_name are referring to the same road,
        assuming we know that both are in the same zip code.'''
    n1 = osm_name.lower(); n2 = nys_name.lower()
    
    # if NYS name is of the form '20A, SOUTH ST'
    L = n2.split(',')
    if len(L) > 1:
        n2 = L[1].strip() # take just the second word and strip
    
    if n1 == n2 or n1 in n2 or n2 in n1:
        return True
    
    # replace any suffix abberviation with the full word
    road_abbrevs = {' st' : ' street', ' rd': ' road', ' dr':' drive',\
                    ' ln':' lane', ' pl':' place', ' ave':' avenue',\
                    ' cir':' circle',' blvd':' boulevard',' pkwy':' parkway',\
                    ' ct':' court',' cres':' crescent',' pk':' park'}
    for key in road_abbrevs:
        n1 = n1.replace(key, road_abbrevs[key])
        n2 = n2.replace(key, road_abbrevs[key])
    
    if n1 == n2 or n1 in n2 or n2 in n1:
        return True
    
    # replace any ordinal abbreviation with the full word
    number_abbrevs = {'1st': 'first', '2nd': 'second', '3rd': 'third',\
                      '4th': 'fourth', '5th': 'fifth', '6th': 'sixth',\
                      '7th': 'seventh', '8th': 'eighth', '9th': 'ninth',\
                      '10th': 'tenth', '11th': 'eleventh', '12th':'twelfth',\
                      '13th': 'thirteenth','14th':'fourteenth','15th':'fifteenth',\
                      '16th': 'sixteenth','17th':'seventeenth','18th':'eighteenth',\
                      '19th': 'nineteenth', '20th':'twentieth'}
    for key in number_abbrevs:
        n1 = n1.replace(key, number_abbrevs[key])
        n2 = n2.replace(key, number_abbrevs[key])
    
    return n1 == n2 or n1 in n2 or n2 in n1

def same_point(p1, p2, tolerance=1):
    ''' Return true if  the UTM coordinates of p1 and p2
        are a total distance of less than 1 away from each other'''
    # use the distance formula
    return math.sqrt((p1[0]-p2[0])**2 + (p1[1] - p2[1])**2) < tolerance

def has_point(point_list, point):
    ''' return True if point_list contains point, using a tolerance of
        1 UTM unit.'''
    for previous_point in point_list:
        if same_point(previous_point, point):
            return True
    return False

def nearest_other_point(L, target):
    ''' Helper function for goes_both_ways. Given list L of ordered pairs, 
        return the point in L that is nearest to target but not equal to it'''
    min_distance = 100000000
    best_point = None
    for point in L:
        dist = math.sqrt((target[0]-point[0])**2 + (target[1] - point[1])**2)
        if dist < min_distance and point != target:
            min_distance = dist
            best_point = point
    return best_point

def angle_between(base_point, point_1, point_2):
    ''' Helper function for goes_both_ways. Let v1 and v2 be the vectors with 
        base at base_point and ending at point_1 and point_2. return the angle 
        between them using dot product'''
    v1 = (point_1[0] - base_point[0], point_1[1] - base_point[1])
    v2 = (point_2[0] - base_point[0], point_2[1] - base_point[1])
    norm_1 = math.sqrt(v1[0]**2 + v1[1]**2)
    norm_2 = math.sqrt(v2[0]**2 + v2[1]**2)
    return math.acos((v1[0]*v2[0] + v1[1]*v2[1]) / (norm_1*norm_2))*180/math.pi

def goes_both_ways(intersection, road):
    ''' return True if road passes through the given intersection coordinates
        rather than ending at it'''
    for point in road['coordinates']:
        if same_point(intersection, point):
            p = point
            break
            
    # get the nearest point to the intersection, other than the intersection itself
    nearest_point = nearest_other_point(road['coordinates'], p)
    
    # check if there is another point on the road that is at least 120 degrees away
    # around the intersection point
    for point in road['coordinates']:
        if not same_point(intersection, point) and not same_point(nearest_point, point):
            if angle_between(intersection, nearest_point, point) > 120:
                return True
    return False

def is_traffic_light(light):
    ''' checks the attributes of the list from the traffic light file
        to make sure that the object is actually a traffic light'''
    return light[-6] == "3-COLOR SIGNAL" and light[-5] == "STANDARD" and\
           light[-1] != None and light[-2] != None

def make_rectangle(points):
    ''' Given a list of ordered pairs of x-y coordinates, returns a 4-tuple of bounding
        values that determine the rectangle containing the line. returns
        (ymax, ymin, xmax, xmin) as North, South, East, West'''
    if len(points) < 2:
        raise ValueError("Not enough points to make a rectangle")
    N = -1000000; S = 1000000; E = -1000000; W = 1000000
    for (x,y) in points:
        N = max(y, N)
        S = min(y, S)
        E = max(x, E)
        W = min(x, W)
    return (N,S,E,W)

def overlap(rec1, rec2):
    ''' return True if the rectangles determined by (N, S, E, W) overlap. This is used 
        when deciding if two roads are close enough to check for an intersection'''
    N1 = rec1[0]; S1 = rec1[1]; E1 = rec1[2]; W1 = rec1[3]
    N2 = rec2[0]; S2 = rec2[1]; E2 = rec2[2]; W2 = rec2[3]
    
    return (S1 <= N2 <= N1 and W1 <= W2 <= E1) or \
           (S1 <= N2 <= N1 and W1 <= E2 <= E1) or \
           (S1 <= S2 <= N1 and W1 <= W2 <= E1) or \
           (S1 <= S2 <= N1 and W1 <= E2 <= E1)

### Zip Code File
The original file contains zip codes for the entire country.  This code writes all of the zip codes that are in Western New York to a new file in the stats folder.

In [5]:
# this takes about 10 seconds

f = open("stats/western_ny_zip_codes.txt", 'w')
sf =  shapefile.Reader("data/zip_codes/cb_2016_us_zcta510_500k")
shape_records = sf.shapeRecords()
for sr in shape_records:
    if sr.record[0][:2] == '14':
        f.write(json.dumps([sr.record[0],convert_to_utm(sr.shape.points)]) + "\n")
f.close() 

# in place of closing the shapefile Reader
sf = None

### AADT File
The original file has traffic data on roads across the entire state. We will write all roads in Livingston County to a separate file in the stats folder.

In [6]:
# this takes about 2 minutes to run

# a rectangle approximating Livingston County
lat_min = 42.5871; lat_max = 42.9846
lon_min = -78.0524; lon_max = -77.4893

# convert the coordinates
xmin, ymin = convert_to_utm([(lon_min, lat_min)])[0]
xmax, ymax = convert_to_utm([(lon_max, lat_max)])[0]

# print them
print("x bounds: ({}, {})".format(xmin,xmax))
print("y bounds: ({}, {})".format(ymin,ymax))

# the file we are writing to
f = open("stats/NYS_livingston.txt", 'w')

# the shapefile we are reading from
sf =  shapefile.Reader("data/AADT_2015_tdv")
shape_records = sf.shapeRecords()

# iterate through each road in the file
for sr in shape_records:
    
    # the list of ordered pairs (x,y) representing points the road passes through
    coordinates = sr.shape.points
    
    # make sure this shape record is actually a road
    if len(coordinates) != 0:
        
        # compute the average point for the line that determines the road
        avg_x = sum([x for (x,y) in coordinates])/len(coordinates)
        avg_y = sum([y for (x,y) in coordinates])/len(coordinates)
        
        # if the average lies within our region, allowing up to 1000 UTM units of error
        if xmin-1000 < avg_x < xmax+1000 and ymin-1000 < avg_y < ymax+1000:
            
            # initialize a dictionary for this road
            road_dict = {}
                        
            # compute the zip code of the road
            zc = get_zip_code(coordinates)
            
            # put the information in the dictionary
            road_dict['name'] = sr.record[5]
            road_dict['AADT'] = sr.record[11]
            road_dict['zip code'] = zc
            road_dict['coordinates'] = coordinates
            
            # manually fix two typos
            if sr.record[5] == "942D, MARY JAMISON":
                road_dict['name'] = 'Mary Jemison Drive'
            if sr.record[5] == "TEMPLE HILL GRO":
                road_dict['name'] = "Temple Hill Street"
            # write [name, AADT, zip code, list of coordinates]
            f.write(json.dumps(road_dict) + "\n")
f.close()

# in place of closing the shapefile Reader
sf = None

x bounds: (249534.3451616777, 297046.00202631776)
y bounds: (4719176.665723138, 4762111.580624357)


### Merging OSM with AADT
This code runs through the OSM file of roads in Livingston County, and then checks through the file we just created of roads with AADT data until it finds a match. For each road, we also compute the bounding rectangle.

In [7]:
# this takes about 10 minutes

# the file we are putting the roads in
f = open("stats/livingston_roads.txt", 'w')

# the file with the NYS data in the local region
g = open("stats/NYS_livingston.txt",'r')
lines = g.read().splitlines()   # store it as a list so we don't have to read from the file every time

# read in the OSM data
sf = shapefile.Reader("data/livingston_shapefile")
shape_records = sf.shapeRecords()
for sr in shape_records:
    rec = sr.record  # this contains all of the important data about the road
    
    # make sure we have a road and not something else
    if (rec[2] in ['primary', 'tertiary', 'secondary', 'residential', 'motorway'])\
        and (rec[1] != '' or "ref" in rec[8]):
        
        # make a dict for the extra information about the road in OSM
        extra_info = split_to_dict(rec[8])
        
        # convert the coordinates to UTM
        coordinates = convert_to_utm(sr.shape.points)
        
        # check if the speed is in the OSM data
        if 'maxspeed' in extra_info:
            speed = int(extra_info['maxspeed'][:2])
        else:
            speed = None
            
        # get the zip code by looking at the mean coordinates.
        zip_code = get_zip_code(coordinates)
        
        # initialize the traffic count to None, until we find it
        traffic = None
        
        # iterate through the NYS data to find the right road, if it's there
        for line in lines:
            NYS_road = json.loads(line)
            nys_zip = NYS_road['zip code']
            
            # if this is the same road
            if zip_code == nys_zip and (same_name(rec[1],NYS_road['name']) or\
             ('ref' in extra_info and NYS_road['name'] in extra_info['ref'])):
                
                # if there is traffic data for it
                if NYS_road['AADT'] != 0.0:
                    traffic = int(NYS_road['AADT'])
                # we have found the match, so don't check the others
                break
        
        # make the bounding rectangle of the road
        rectangle = make_rectangle(coordinates)
        
        # put everything into a dictionary
        road = {'name':rec[1], 'zip code': zip_code, 'coordinates': coordinates,\
                'AADT': traffic, 'speed': speed, 'road type':rec[2], 'rectangle': rectangle}
        f.write(json.dumps(road) + "\n")
f.close()
g.close()

# in place of closing the shapefile Reader
sf = None

### Intersections

In [8]:
# takes 9 minutes

# adjust this boolean to determine whether to run the code or not
intersections_generated = False
if not intersections_generated:
    f = open("stats/livingston_roads.txt", 'r')
    g = f.read().splitlines()
    f.close()
    roads = []
    for x in g:
        roads.append(json.loads(x))
    intersections = {}
    # each intersection is {coordinates: {'road_list': list of roads, 'light':  0 or 1}}

    # iterate over all roads
    for road in roads:

        # iterate over all points in the road
        for point in road['coordinates']:

            # if this point is already in an intersection
            if has_point(list(intersections.keys()), point):

                # if it is from a new road
                if not road in intersections[tuple(point)]['road_list']:
                    intersections[tuple(point)]['road_list'].append(road)

            # if this point is a new point
            else: 
                # iterate over the other roads
                for other_road in roads:

                    # make sure it's a different road
                    if other_road != road:
                        
                        # check if the rectangles overlap
                        if overlap(road['rectangle'], other_road['rectangle']):

                            for other_point in other_road['coordinates']:

                                # if we have a new intersection
                                if same_point(other_point, point):

                                    intersection = {'road_list': [road, other_road], 'light' : 0}
                                    # add the intersection to the dictionary
                                    intersections[tuple(point)] = intersection
    f = open("stats/livingston_intersections.txt", 'w')
    for intsec in intersections:
        f.write(json.dumps([intsec,intersections[intsec]]) + '\n')
    f.close()

### Traffic Lights
For each intersection in our file, we now compute a few statistics about it and also check through the traffic light file to see if there is one at the intersection.

In [14]:
# takes 2 minutes

f = open("stats/livingston_intersections.txt",'r')

# a new file to hold all of the calculations for each intersection
g = open("stats/livingston_intersections_plus.txt", 'w')

h = open("data/traffic_lights.txt",'r')
s = ""
for line in h:
    s += line
h.close()
traffic_lights = json.loads(s)['data']


for line in f:
    intersection = json.loads(line)
    
    # the coordinates of the point
    intsec_point = intersection[0]
    
    roads = intersection[1]['road_list']
    
    # the number of directions at the intersection is initially the number of roads
    num_directions = len(roads)
    
    # the total amount of traffic from all the roads
    AADTs = []
    
    # giving a score to the road types
    road_type_score = {'residential': 1, 'tertiary': 2, 'secondary': 3, 'primary': 4, 'motorway': 5}
    road_type_sum = 0
    num_types = 0  # need this counter to take an average
    
    speed_sum = 0
    num_speeds = 0 # need this counter to take an average
    
    for road in roads:
        
        # store the boolean deciding if the road goes both ways,
        # since we will need it multiple times
        both_ways = goes_both_ways(intsec_point, road)
        
        # if the road is a state road
        if road['AADT'] != None:
            AADTs.append(road['AADT'])
            
            # if the road goes both way, we have another direction
            if both_ways:
                num_directions += 1
                AADTs.append(road['AADT'])
                
        # if the road is a local road
        else:
            # if the road goes both way, we have another direction
            if both_ways:
                num_directions += 1
                
        # for every road
        if road['road type'] != None:
            road_type_sum += road_type_score[road['road type']]
            num_types += 1
            if both_ways:
                road_type_sum += road_type_score[road['road type']]
                num_types += 1
        
        if road['speed'] != None:
            speed_sum += road['speed']
            num_speeds += 1
            if both_ways:
                speed_sum += road['speed']
                num_speeds += 1
    
    AADT_sum = sum(AADTs)
    num_state_directions = len(AADTs)
    if num_state_directions != 0:
        mean_value = AADT_sum/num_state_directions
        stdv = math.sqrt(sum([(v - mean_value)**2 for v in AADTs])/num_state_directions)
    else:
        stdv = None
    
    # calculate the averages
    try:
        average_AADT = AADT_sum/num_state_directions
    except ZeroDivisionError:
        average_AADT = None
    try:
        average_road_score = road_type_sum/num_types
    except ZeroDivisionError:
        average_road_score = None
    try:
        average_speed = speed_sum/num_speeds
    except ZeroDivisionError:
        average_speed = None
        
    traffic_light = 0
    # check if there is a traffic light
    for light in traffic_lights:
        
        # use a tolerance of 8 to check if the points are the same
        if is_traffic_light(light) and \
        same_point((float(light[-1]), float(light[-2])), intsec_point,10):
            traffic_light = 1
            break # stop searching if we found one
           
    # make it None to denote missing data
    if AADT_sum == 0:
        AADT_sum = None
        
    # a new dictionary to store the statistics
    int_plus = {'coordinates': intsec_point, 'num directions': num_directions,\
                'num state directions': num_state_directions, 'average AADT': average_AADT,\
                'total AADT': AADT_sum, 'average road score': average_road_score,\
                'average speed': average_speed, 'traffic light': traffic_light,\
                'AADT stdv': stdv}
    g.write(json.dumps(int_plus) + '\n')
    
f.close()
g.close()      

### Marginal Mean Imputation on AADT

In [16]:

road_type_averages = {'residential': 1348.691, 'tertiary': 944.299, 'secondary': 3293.608,\
                      'primary': 5536.308, 'motorway': 14052.186}

# this file contains the actual roads for each intersection
f = open("stats/livingston_intersections.txt",'r')

# a new file to hold all of the calculations for each intersection with imputed AADT
g = open("stats/livingston_intersections_mmi.txt", 'w')

h = open("data/traffic_lights.txt",'r')
s = ""
for line in h:
    s += line
h.close()
traffic_lights = json.loads(s)['data']


for line in f:
    intersection = json.loads(line)
    
    # the coordinates of the point
    intsec_point = intersection[0]
    
    roads = intersection[1]['road_list']
    
    # the number of directions at the intersection is initially the number of roads
    num_directions = len(roads)
    
    # the total amount of traffic from all the roads
    AADTs = []
    
    road_type_sum = 0
    num_types = 0  # need this counter to take an average
    
    speed_sum = 0
    num_speeds = 0 # need this counter to take an average
    
    for road in roads:
        
        # store the boolean deciding if the road goes both ways,
        # since we will need it multiple times
        both_ways = goes_both_ways(intsec_point, road)
        
        # if the road is a state road
        if road['AADT'] != None:
            AADTs.append(road['AADT'])
            
            # if the road goes both way, we have another direction
            if both_ways:
                num_directions += 1
                AADTs.append(road['AADT'])
                
        # if the road is a local road
        else:
            AADTs.append(road_type_averages[road['road type']])    # the imputed AADT value
            # if the road goes both ways, we have another direction
            if both_ways:
                num_directions += 1
                AADTs.append(road_type_averages[road['road type']])# the imputed AADT value
                
        # for every road
        if road['speed'] != None:
            speed_sum += road['speed']
            num_speeds += 1
            if both_ways:
                speed_sum += road['speed']
                num_speeds += 1
    
    AADT_sum = sum(AADTs)
    num_state_directions = len(AADTs)
    if num_state_directions != 0:
        mean_value = AADT_sum/num_state_directions
        stdv = math.sqrt(sum([(v - mean_value)**2 for v in AADTs])/num_state_directions)
    else:
        stdv = None
    
    # calculate the averages
    try:
        average_AADT = AADT_sum/num_state_directions
    except ZeroDivisionError:
        average_AADT = None
    try:
        average_speed = speed_sum/num_speeds
    except ZeroDivisionError:
        average_speed = None
        
    traffic_light = 0
    # check if there is a traffic light
    for light in traffic_lights:
        
        # use a tolerance of 8 to check if the points are the same
        if is_traffic_light(light) and \
        same_point((float(light[-1]), float(light[-2])), intsec_point,10):
            traffic_light = 1
            break # stop searching if we found one
           
    # make it None to denote missing data
    if AADT_sum == 0:
        AADT_sum = None
        
    # a new dictionary to store the statistics
    int_plus = {'coordinates': intsec_point, 'num directions': num_directions,\
                'num state directions': num_state_directions, 'average AADT': average_AADT,\
                'total AADT': AADT_sum,\
                'average speed': average_speed, 'traffic light': traffic_light,\
                'AADT stdv': stdv}
    g.write(json.dumps(int_plus) + '\n')
    
f.close()
g.close()